# Chest Cancer Classification

## Load the data

{'Content-Type': 'text/html; charset=utf-8', 'Date': 'Fri, 02 Jun 2023 17:03:16 GMT', 'Cache-Control': 'no-cache, no-store', 'Content-Encoding': 'gzip', 'Pragma': 'no-cache', 'Set-Cookie': 'CSRF-TOKEN=CfDJ8KCSgn9T1oROnY3CWS6bDdwE7HgEnVtIJgIPFDI_taF0wMUPo84ovuo8yvwE5ATHNz-e4fJtyRtyHqBeL5PpKwIUYgLz7ZEzzCnjag_CGw; path=/; secure; samesite=lax; httponly, XSRF-TOKEN=CfDJ8KCSgn9T1oROnY3CWS6bDdw4GHWqBBiQ--o4Oi9OMlN-rKHX832sw2Rnv3cahY-LPyO2_DnC6dRKMQWUw2tTXEo_sLS53fYUGdoDEsVz4fjc3A; path=/; secure; samesite=lax, CLIENT-TOKEN=eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJrYWdnbGUiLCJhdWQiOiJjbGllbnQiLCJzdWIiOm51bGwsIm5idCI6IjIwMjMtMDYtMDJUMTc6MDM6MTcuMjU4MTg5NloiLCJpYXQiOiIyMDIzLTA2LTAyVDE3OjAzOjE3LjI1ODE4OTZaIiwianRpIjoiMzk3YzkyMDgtNGQ4Yy00MzQzLWFlNmItNGU4N2E4MWRkYTg2IiwiZXhwIjoiMjAyMy0wNy0wMlQxNzowMzoxNy4yNTgxODk2WiIsImFub24iOnRydWUsImZmIjpbIktlcm5lbHNEaXNhYmxlVW51c2VkQWNjZWxlcmF0b3JXYXJuaW5nIiwiS2VybmVsc0ZpcmViYXNlTG9uZ1BvbGxpbmciLCJDb21tdW5pdHlMb3dlckhlYWRlclNpemVzIiwiQWxsb3dGb3J1bUF0dGFja